# Demo Code: Export&Import Selected Assets to a zip file

Actually, every selected assets have dedicated command. Highly recommended to use dedicated one unless you really want to put them togehter.

## Prerequisite
1. Create your own yaml based on [configuration_template.yaml](./configuration_template.yaml), and modify "CONFIG_FILE" below.


In [1]:
import sys
import os
import yaml
import subprocess

In [2]:
# point to a local file with credentials. It is not synced to git.
CONFIG_FILE = "export_import_selected_assets.yaml"

In [3]:
# Load parameters from the YAML file
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DSJOB_URL = config['url']
DSJOB_USER = config['user']
DSJOB_PWD = config['password']
EXPORT_PRJ_NAME = config['export_prj_name']
if 'export_zip_name' in config:
    EXPORT_ZIP_NAME = config['export_zip_name']
else:
    EXPORT_ZIP_NAME = EXPORT_PRJ_NAME+"_assets.zip"
    
datastages =   config['datastages']
pipelines =  config['pipelines']
testcases = config['testcases']

IMPORT_PRJ_NAME = config['import_prj_name']
print("EXPORT_PRJ_NAME=",EXPORT_PRJ_NAME)
print("IMPORT_PRJ_NAME=",IMPORT_PRJ_NAME)

EXPORT_PRJ_NAME= Multicloud Data Integration L3 Tech Lab
IMPORT_PRJ_NAME= DataStage Import


In [4]:
# Enable dsjob
%env CPDCTL_ENABLE_DSJOB=true
%env CPDCTL_ENABLE_DATASTAGE=true
%env CPDCTL_ENABLE_VOLUMES=1
!echo $CPDCTL_ENABLE_DSJOB
!echo $CPDCTL_ENABLE_DATASTAGE
!echo $CPDCTL_ENABLE_VOLUMES

env: CPDCTL_ENABLE_DSJOB=true
env: CPDCTL_ENABLE_DATASTAGE=true
env: CPDCTL_ENABLE_VOLUMES=1
true
true
1


In [5]:
# Configure cpdctl with the parameters
!cpdctl config user set CP4D-user --username "$DSJOB_USER" --password "$DSJOB_PWD"
!cpdctl config profile set CP4D-profile --url "$DSJOB_URL" --user CP4D-user
!cpdctl config profile use CP4D-profile

Switched to profile "CP4D-profile".


In [6]:
# list all projects
!cpdctl dsjob list-projects

...
Business Catalog Enrichment
DataStage Import1
GDPR Lineage
test_python_pipeline
DataStage Import
DQ Definition_Rules
Multicloud Data Integration L3 Tech Lab
Data Replication Lab
Auto Policy Risk
evaluate-an-ml-model
Data-Science-and-MLOps

Total: 11 Projects

Status code = 0


In [7]:
# Format the string for the export-zip
datastages_str=''
pipelines_str=''
testcases_str=''
if len(datastages)>0:
    datastages_str = ' '.join(f'--name "{datastage}"' for datastage in datastages)
if len(pipelines)>0:
    pipelines_str = ' '.join(f'--pipeline "{pipeline}"' for pipeline in pipelines)
if len(testcases)>0:
    testcases_str = ' '.join(f'--name "{testcases}"' for testcase in testcases)

cmd_str = f'cpdctl dsjob export-zip --project "{EXPORT_PRJ_NAME}" \
{datastages_str} {pipelines_str} {testcases_str} --file-name "{EXPORT_ZIP_NAME}"'

print(cmd_str)

cpdctl dsjob export-zip --project "Multicloud Data Integration L3 Tech Lab" --name "Employee Ranking" --pipeline "Pipeline1" --pipeline "Pipeline2"  --file-name "own_name.zip"


In [8]:
os.system(cmd_str)

...

Status code =  0


0

## Import to a project with **import-zip**

Connection between development and production environment are usually different. Hence we usually skip "connection" with "--skip-on-replace connection" flag.

In [9]:
!cpdctl dsjob import-zip --project "$IMPORT_PRJ_NAME" --file-name "$EXPORT_ZIP_NAME" --conflict-resolution replace --skip-on-replace connection --wait 200 

...
2024-06-03 21:32:16: Waiting until import finishes, import id: 2e14fc12-e8aa-42bf-bede-e07eaebe2000
2024-06-03 21:32:26: Project import status: started,  total: 5, completed: 3, failed: 0, skipped: 2.
2024-06-03 21:32:46: Project import status: completed,  total: 5, completed: 3, failed: 0, skipped: 2.
Information:
	Connection: Data Virtualization,	  New connection is exactly the same as an existing connection, resource is not updated.

	Connection: Data Warehouse,	  New connection is exactly the same as an existing connection, resource is not updated.


Status code =  0
